In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Laoding meat consumption data
df = pd.read_csv('./HIGH_AGLINK_2017_30052018101303688.csv', header=0)
# to meet the country name to WordBank data
df.reset_index(inplace=True)
df.loc[df['Country'] == 'European Union-28', 'Country'] = 'European Union'
df.loc[df['Country'] == 'Republic of South Africa', 'Country'] = 'South Africa'
df.loc[df['Country'] == 'Viet Nam', 'Country'] = 'Vietnam'

In [3]:
# Norway and Switzerland population is empty so we need to load from UN population data
pf = pd.read_excel('./population(switzandnorway).xlsx', 'Data', header=0)

In [4]:
# Getting variables from World Bank dataset
cf = pd.read_excel('./variables.xlsx', 'data')
cf.loc[cf['Country Name'] == 'Egypt, Arab Rep.', 'Country Name'] = 'Egypt'
cf.loc[cf['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'Korea'
cf.loc[cf['Country Name'] == 'Iran, Islamic Rep.', 'Country Name'] = 'Iran'
cf.drop(['Country Code', 'Series Code'], axis=1, inplace=True)
cf = cf.set_index(['Country Name', 'Series Name']).unstack(0).T
cf.index.rename('Time', 0, inplace=True)
cf.rename(columns={'Country Name': 'Country'}, inplace=True)

In [5]:
# religion data triming for merge
uf = pd.read_excel('./UNdata_Export.xlsx', 'data')
uf.rename(columns={'Country or Area': 'Country', 'Year': 'Time'}, inplace=True)
uf = uf.pivot_table(index=['Country', 'Time'], columns=['Area', 'Sex', 'Religion'], values='Value')
uf.columns = ['_'.join(x) for x in uf.columns.tolist()]
uf.reset_index(inplace=True)
# To rename on some countries for merging with other data
for k, i in {'Viet Nam': 'Vietnam', 'Republic of Korea': 'Korea', 'Iran (Islamic Republic of)': 'Iran', 
             'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'}.items():
    uf.loc[uf['Country'] == k, 'Country'] = i
    
uf.set_index(['Country', 'Time'], inplace=True)

In [6]:
def triming_by_meatcategory(df, category):
    """
    Triming for each meat category (we have four categories now.)
    """
    bf = df[df.Commodity == category][['Country', 'Variable', 'Time', 'Value']]
    bf = bf.pivot_table(index=['Country', 'Time'], columns='Variable', values='Value')
    bf.drop(['Producer price', 'World Price'], axis=1, inplace=True)
    # To remove regional groups(e.g. OECD and so on)
    bf.drop(index=[x for x in bf.index.get_level_values(0).unique() if x.isupper()], inplace=True)
    bf.drop(index=['OECD countries', 'Non-OECD'], inplace=True)
    
    bf.loc['Norway']['Human consumption per capita'] = bf.loc['Norway']['Consumption']*1000 / pf.set_index('Country').iloc[0]
    bf.loc['Switzerland']['Human consumption per capita'] = bf.loc['Switzerland']['Consumption']*1000 / pf.set_index('Country').iloc[1]
    
    return bf

In [7]:
bf = triming_by_meatcategory(df, 'Beef and veal (cwe)')

In [8]:
# for check coutries' name match between dataframes
[x for x in bf.index.get_level_values(0).unique() if x not in cf.index.get_level_values(1).unique()]

[]

In [9]:
# To divide religion data by population, we need to load pop db from UN
pf = pd.read_excel('./TotalPopSex-20180606042034.xlsx', 'Data', skiprows=1).drop(['ISO 3166-1 numeric code', 'Note'], axis=1)
pf.Location = pf.Location.apply(lambda x: x.strip())
pf.rename(columns={'Location': 'Country'}, inplace=True)
for k, i in {'Iran (Islamic Republic of)': 'Iran', 'Republic of Korea': 'Korea', 'Viet Nam': 'Vietnam'}.items():
    pf.loc[pf['Country'] == k, 'Country'] = i
pf.set_index('Country', inplace=True)

In [10]:
# To drop unable countries according to the population dataset
uf.drop(index=[x for x in uf.index.get_level_values(0).unique() if x not in pf.index.unique()], inplace=True,)
# To remove unnecessary countries on religion data
mask = pf.groupby(pf.index).count()
nosexdata = mask[mask.Sex < 3].index
uf.drop(index=nosexdata, inplace=True, errors='ignore')

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [11]:
# To get the percentage of religion by population by country and sex
regexdict = {'Both_*': 'Both sexes combined', 'Male_*': 'Male', 'Female_*': 'Female'}
flist = []
for k, i in regexdict.items():
    rf = uf.filter(regex=k)
    for c, t in rf.index:
        rf.loc[c, t] = rf.loc[c, t].divide(pf[pf.Sex == i].loc[c, t] * 1000)
        
    flist.append(rf)
    
uf = pd.concat(flist, axis=1,)

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [15]:
# To delete unnecessary variables in memory
del rf, pf, c, flist, i, k, mask, nosexdata, regexdict, t

In [14]:
whos

Variable                  Type         Data/Info
------------------------------------------------
bf                        DataFrame    Variable      Consumption<...>\n[1628 rows x 6 columns]
cf                        DataFrame    Series Name              <...>[7392 rows x 118 columns]
df                        DataFrame           index COUNTRY     <...>[54684 rows x 12 columns]
np                        module       <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd                        module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
plt                       module       <module 'matplotlib.pyplo<...>\\matplotlib\\pyplot.py'>
sns                       module       <module 'seaborn' from 'C<...>s\\seaborn\\__init__.py'>
triming_by_meatcategory   function     <function triming_by_meat<...>ry at 0x0000029CF9975E18>
uf                        DataFrame                             <...>[151 rows x 3154 columns]


## From this line, its on test

In [35]:
len(bf.index.get_level_values(1).unique())

37

In [38]:
len(uf.index.get_level_values(0).unique())

91

In [53]:
uf.shape

(151, 3154)

In [50]:
len(uf.columns) / 9

350.44444444444446

In [17]:
[x for x in bf.index.get_level_values(0).unique() if x not in uf.index.get_level_values(0).unique()]

['Algeria',
 'Argentina',
 'China',
 'Colombia',
 'Egypt',
 'European Union',
 'Haiti',
 'Japan',
 'Nigeria',
 'Norway',
 'Pakistan',
 'Russian Federation',
 'Saudi Arabia',
 'Sudan',
 'Tanzania',
 'Turkey',
 'Ukraine',
 'United States',
 'Uruguay']

In [36]:
bf.reset_index(inplace=True)
bf.set_index(['Country', 'Time'], inplace=True)

In [44]:
sf = pd.merge(bf, cf, on=['Country', 'Time'])